## WFS is a method for parsing and downloading Geographic Features to be used in analyses and graphs  

In this example, we're going to look at some layers that are currently accessible on our instance of PAVICS running locally. With WFS, we can see what is available, collect the layers we want by using a query, download the results in GML, and visualize them using OGR.

We begin by loading the libraries needed for parsing and downloading from WFS and for opening and visualizing the results

In [1]:
%matplotlib inline
# Import WFS from owslib
from owslib.wfs import WebFeatureService

# Import ogr from osgeo in order to work with vector data
from osgeo import ogr

# Import matplotlib for visualizing the outputs
import matplotlib.path as mpath
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt

import numpy as np
import shutil

ImportError: libpoppler.so.76: cannot open shared object file: No such file or directory

We start by making a connection to the PAVICS instance we have locally on our server, `Boreas`. Using WFS, we can very quickly see the contents, which are the layers and the workspaces they're located with (ie: TravisTest, scratchTJS). These layer names act as dictionaries for 

In [ ]:
wfs_url = 'http://boreas.ouranos.ca/geoserver/wfs'
# connection
wfs = WebFeatureService(wfs_url, version='2.0.0')

for i in wfs.contents:
    print(i)

If we want to know more about these layers, we can use the layer names as keys in a dictionary to return more information from each layer.

In [ ]:
for layerID in wfs.contents.keys():
    layer = wfs[layerID]
    print('Layer ID:', layerID)
    print('Title:', layer.title)
    print('Boundaries:', layer.boundingBoxWGS84, '\n')

We can then perform a GetFeatures call using the layer name as a target. This returns an IOstream that can be written as a GML file, a common file format for vector data served throughout the web.

In [ ]:
layer_io = wfs.getfeature(typename='TravisTest:region_admin_poly')

# We can also choose to do a bounding box subset within our GetFeature call!
# layer_io = wfs.getfeature(typename='TravisTest:region_admin_poly', bbox=(-74.5, 45.2, -73, 46))

data_file = 'output.gml'

try:
    with open(data_file, 'w', encoding="utf-8") as f:
        layer_io.seek(0)
        shutil.copyfileobj(layer_io, f)
        print('File successfully downloaded!')
except:
    print('File failed to download.')


Once the GML is downloaded, we can either open it with a GIS application or we can read the features in using OGR, a library for manipulating vector data. The following code reads the number of features within the GML and outputs the each feature's names and values.

In [ ]:
data  = ogr.Open(data_file, 0)

if data is None:
    print("File failed to be read!")
else:
    layer = data.GetLayer()
    featureCount = layer.GetFeatureCount()
    print("Number of features in {}: {}".format(data_file, featureCount), '\n')
    
    layerDefn = layer.GetLayerDefn()
    
    print('Layer Name:', layerDefn.GetName())
    
    for feature in layer:
        for i in range(layerDefn.GetFieldCount()):
            fieldname = layerDefn.GetFieldDefn(i).GetName()
            print(fieldname, ' Value:', feature.GetField(fieldname))
        

This `layer` object can also be examined with matplotlib if we want to create maps or figures within Python without a Grapgical User Interface.


In [ ]:
ext = layer.GetExtent()
xoff = (ext[1]-ext[0])/50
yoff = (ext[3]-ext[2])/50


plt.ioff()
plt.plot()
ax = plt.gca()
ax.set_xlim(ext[0]-xoff,ext[1]+xoff)
ax.set_ylim(ext[2]-yoff,ext[3]+yoff)

paths = []
layer.ResetReading()

# Read all features in layer and store as paths
for feat in layer:

    for geom in feat.GetGeometryRef():
        envelope = np.array(geom.GetEnvelope())
        # check if geom is polygon
        if geom.GetGeometryType() == ogr.wkbPolygon:
            codes = []
            all_x = []
            all_y = []
            for i in range(geom.GetGeometryCount()):
                # Read ring geometry and create path
                r = geom.GetGeometryRef(i)
                x = [r.GetX(j) for j in range(r.GetPointCount())]
                y = [r.GetY(j) for j in range(r.GetPointCount())]
                # skip boundary between individual rings
                codes += [mpath.Path.MOVETO] + \
                             (len(x)-1)*[mpath.Path.LINETO]
                all_x += x
                all_y += y
            path = mpath.Path(np.column_stack((all_x,all_y)), codes)
            paths.append(path)
    # Add paths as patches to axes
    for path in paths:
        patch = mpatches.PathPatch(path, \
                facecolor='0.8', edgecolor='cyan')
        ax.add_patch(patch)



ax.set_aspect(1.0)
plt.show()